In [ ]:
import asyncio
from ai_agent.agent.service import GenericAgent
from ai_agent.agent.context import SimpleExecutionContext
from ai_agent.controller.service import Controller
from ai_agent.filesystem import FileSystem
from ai_agent.llm.factory import create_llm
from ai_agent.llm.views import LLMType
from ai_agent.actions.registry_helper import register_all_actions

async def main():
    # Set up components
    llm = create_llm(LLMType.ANTHROPIC, model='claude-3-5-sonnet-20241022')
    file_system = FileSystem()
    controller = Controller()
    
    # Register built-in actions
    register_all_actions(controller.registry, file_system)
    
    # Create execution context
    execution_context = SimpleExecutionContext(
        file_system=file_system,
        llm=llm,
        registry=controller.registry,
    )
    
    # Create and run agent
    agent = GenericAgent(
        task="create a html file with snake game with sleak ui. you allowed only to generate html file. You should write code, then evaluate it, then fix bugs. Do it in multiple steps.",
        llm=llm,
        execution_context=execution_context,
        controller=controller,
        file_system=file_system,
    )
    
    history = await agent.run(max_steps=10)
    if history.last and history.last.model_output:
        print(f"Task completed: {history.last.model_output.is_done}")
    else:
        print("Agent failed to complete the task")

await main()

📋 Available actions: 14 total
DEBUG: Tool input received: {'current_state': {'thinking': "Let's create a snake game in HTML with a sleek UI. We'll do this in multiple steps:\n1. First, create the initial HTML file with basic structure and CSS\n2. Add the game canvas and JavaScript logic\n3. Test and evaluate the code\n4. Fix any bugs and improve the UI", 'evaluation_previous_goal': None, 'memory': 'Starting the snake game development. Will create a complete game with canvas, score display, and modern UI.', 'next_goal': 'Create the initial HTML file with basic structure, styling, and game canvas setup'}, 'action': [{'write_file': {'path': 'snake.html', 'content': '<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Sleek Snake Game</title>\n    <style>\n        body {\n            margin: 0;\n            padding: 0;\n            display: flex;\n            justify-content: center;\n